In [112]:
import os
import pandas as pd
import numpy as np

In [96]:
path = './customer_db/src/'

customer = pd.read_csv(path + 'customer.csv', sep=';')

purchase = pd.read_csv(path + 'purchase.csv', sep=';')
product = pd.read_csv(path + 'product.csv', sep=';').assign(new_price=lambda x: [float(price) if price != 'Free' else 0 for price in x.price])
product_feedback = pd.read_csv(path + 'product_feedback.csv', sep=';')
service_feedback = pd.read_csv(path + 'service_feedback.csv', sep=';')


In [186]:
f3 = (
    purchase
    .merge(
        product[['id', 'new_price']].rename(columns={'id':'product_id'}), 
        on='product_id', 
        how='left')
    .assign(revenue=lambda x: x.new_price * x.quantity)
    .merge(
        product_feedback[['purchase_id', 'rating']]
        .rename(columns={'purchase_id':'id'}), 
        on='id', 
        how='left')
    .fillna(0)
    .groupby('user_id').agg({'id':'count', 'revenue':'sum', 'rating': 'mean'})
)
f3.columns=['purchases_num', 'total_spendings', 'mean_rating_by_prod']
#[, 'mostProduct', 'mostCategory', 'feedbacks']

In [162]:
most_fav_prod = (
    purchase
    .merge(
        product_feedback[['purchase_id', 'rating']]
        .rename(columns={'purchase_id':'id'}), 
        on='id', 
        how='left')
    .merge(
        product[['id', 'new_price', 'product_name', 'category_name']].rename(columns={'id':'product_id'}), 
        on='product_id', 
        how='left')
    .fillna(0)
    .groupby(['user_id', 'product_name'])
    .agg(max=('rating', 'max'))
    .reset_index()
    .sort_values(['user_id', 'max', 'product_name'], ascending=[True, False, True])
    .drop_duplicates(['user_id'],keep='first')[['user_id', 'product_name']]
)

most_fav_cat = (
    purchase
    .merge(
        product_feedback[['purchase_id', 'rating']]
        .rename(columns={'purchase_id':'id'}), 
        on='id', 
        how='left')
    .merge(
        product[['id', 'new_price', 'product_name', 'category_name']].rename(columns={'id':'product_id'}), 
        on='product_id', 
        how='left')
    .fillna(0)
    .groupby(['user_id', 'category_name'])
    .agg(count=('category_name', 'count'))
    .reset_index()
    .sort_values(['user_id', 'count', 'category_name'], ascending=[True, False, True])
    .drop_duplicates(['user_id'],keep='first')[['user_id', 'category_name']]
)

sfb = service_feedback.groupby('customer_id').agg(service_fb=('rating', 'count')).reset_index().rename(columns={'customer_id':'user_id'})
pfb = (
    purchase
    .merge(
        product[['id', 'new_price']].rename(columns={'id':'product_id'}), 
        on='product_id', 
        how='left')
    .assign(revenue=lambda x: x.new_price * x.quantity)
    .merge(
        product_feedback[['purchase_id', 'rating']]
        .rename(columns={'purchase_id':'id'}), 
        on='id', 
        how='left')
    .fillna(0)
    .query('rating != 0')
    .groupby('user_id').agg(product_fb=('rating', 'count'))
)
total_fb = pfb.merge(sfb, on='user_id', how='outer').fillna(0).assign(total_fb=lambda x: x.product_fb + x.service_fb)[['user_id', 'total_fb']]

In [193]:
(
    customer[['id']]
    .rename(columns={'id':'user_id'})
    .merge(f3, on='user_id', how='left')
    .merge(most_fav_prod, on='user_id', how='left')
    .merge(most_fav_cat, on='user_id', how='left')
    .merge(total_fb, on='user_id', how='left')
)

,user_id,purchases_num,total_spendings,mean_rating_by_prod,product_name,category_name,total_fb
0,1,11.0,563.84,4.363636,Animal Crossing: New Horizons,Games,7.0
1,2,10.0,398.34,6.500000,A Game of Thrones,Movies,9.0
2,3,10.0,247.85,3.500000,Crime and Punishment,Movies,5.0
3,4,14.0,359.27,4.357143,Brave New World,Games,9.0
4,5,9.0,164.91,3.000000,Celeste,Books,4.0
5,6,7.0,478.39,3.857143,Splatoon 2,Games,4.0
6,7,7.0,161.89,4.285714,Hollow Knight,Books,4.0
7,8,13.0,433.32,4.846154,Fall Guys,Books,10.0
8,9,15.0,337.29,3.666667,Stardew Valley,Movies,8.0
9,10,10.0,267.85,2.400000,Mortal Kombat 11,Books,4.0
